# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [2]:
data = run_engine(data)

100%|███████████████████████████████| 109001/109001 [00:09<00:00, 10948.63it/s]

pl    0.01409
dtype: float64
--------------------------------------
             DateTime_frmt          close_type       pl
99310  2021-01-04 19:18:22  simple_take_profit  0.00017
100881 2021-01-04 19:51:12  simple_take_profit  0.00023
101036 2021-01-04 19:53:41  simple_take_profit  0.00025
101789 2021-01-04 20:03:05  simple_take_profit  0.00031
102543 2021-01-04 20:12:56  simple_take_profit  0.00012
103585 2021-01-04 20:35:35         simple_stop -0.00046
103833 2021-01-04 20:40:27  simple_take_profit  0.00023
104199 2021-01-04 20:49:07  simple_take_profit  0.00024
105242 2021-01-04 21:07:45  simple_take_profit  0.00030
106052 2021-01-04 21:38:06  simple_take_profit  0.00033
106883 2021-01-04 22:00:40  simple_take_profit  0.00045
106940 2021-01-04 22:06:19  simple_take_profit  0.00051
107093 2021-01-04 22:18:19  simple_take_profit  0.00028
108132 2021-01-04 23:27:26  simple_take_profit  0.00026
108819 2021-01-04 23:51:05  simple_take_profit  0.00017


In [3]:
generate_result_report(data)

--------------------------------------
Run Complete : data/ema_df-(2021-2021)-(1-1)-(4-4).csv
--------------------------------------


--------------------------------------
Net PL
0.0141
--------------------------------------


--------------------------------------
Monthly PL
                pl
month_val         
1          0.01409
--------------------------------------


--------------------------------------
Count of transactions by Close Type
..................
                     pl
close_type             
simple_stop          32
simple_take_profit  115
--------------------------------------


--------------------------------------
Mean value of transactions by Close Type
..................
                          pl
close_type                  
simple_stop        -0.000534
simple_take_profit  0.000271
--------------------------------------


--------------------------------------
Sum by Close Type
..................
                         pl
close_type                 
simp